# American options tree

Listo. Mostré tres tablas:

* **Futures price lattice** $F(t,j)$ para vencimiento en el paso 15, filas $t=0..10$.
* **Valor del call americano** $V(t,j)$ para $t=0..10$.
* **Decisión** en cada nodo: **E** = ejercer, **C** = continuar, **E/C** = indiferente.

Según el árbol, el **primer** periodo donde aparece un nodo con ejercer/indiferencia es el **t=4** en esta calibración. Si necesitas el gráfico del árbol en forma de diagrama, te lo genero.


In [3]:
# Rebuild lattices and display them
import numpy as np
import pandas as pd
from math import exp, sqrt
# from caas_jupyter_tools import display_dataframe_to_user

S0 = 100.0
r  = 0.02
c  = 0.01
sigma = 0.30
T  = 0.25
N  = 15
NF = 15
NO = 10
K  = 110.0

dt = T / N
R  = np.exp(r*dt)
u  = np.exp(sigma*np.sqrt(dt))
d  = 1.0/u
q  = (np.exp((r-c)*dt) - d) / (u - d)

S = np.zeros((N+1, N+1))
S[0,0] = S0
for i in range(1, N+1):
    S[i,0] = S[i-1,0]*d
    for j in range(1, i+1):
        S[i,j] = S[i-1,j-1]*u

carry = np.exp((r - c) * dt)
F = np.zeros_like(S)
for i in range(NF+1):
    for j in range(i+1):
        F[i,j] = S[i,j] * (carry ** (NF - i))

rows = []
for i in range(0, NO+1):
    row = [round(F[i,j], 6) for j in range(i+1)]
    rows.append(row + [""]*(NO+1 - len(row)))
F_df = pd.DataFrame(rows, columns=[f"j={j}" for j in range(NO+1)])
F_df.index = [f"t={i}" for i in range(NO+1)]

# Backward induction
V = [[None]*(i+1) for i in range(NO+1)]
decision = [[""]*(i+1) for i in range(NO+1)]
for j in range(NO+1):
    V[NO][j] = max(F[NO,j] - K, 0.0)
    decision[NO][j] = "T"
for i in range(NO-1, -1, -1):
    for j in range(i+1):
        cont = (q*V[i+1][j+1] + (1-q)*V[i+1][j]) / R
        intr = max(F[i,j] - K, 0.0)
        if intr > cont + 1e-12:
            V[i][j] = intr; decision[i][j] = "E"
        elif abs(intr - cont) <= 1e-12:
            V[i][j] = intr; decision[i][j] = "E/C"
        else:
            V[i][j] = cont; decision[i][j] = "C"

V_rows = []
D_rows = []
for i in range(0, NO+1):
    vrow = [round(V[i][j], 6) for j in range(i+1)]
    drow = [decision[i][j] for j in range(i+1)]
    V_rows.append(vrow + [""]*(NO+1 - len(vrow)))
    D_rows.append(drow + [""]*(NO+1 - len(drow)))

V_df = pd.DataFrame(V_rows, columns=[f"j={j}" for j in range(NO+1)])
V_df.index = [f"t={i}" for i in range(NO+1)]
D_df = pd.DataFrame(D_rows, columns=[f"j={j}" for j in range(NO+1)])
D_df.index = [f"t={i}" for i in range(NO+1)]

from IPython.display import display, Markdown

display(Markdown("**Futures price lattice F(t,j) for maturity at step 15 (rows t=0..10):**"))
display(F_df)
display(Markdown("**American call on futures: value lattice V(t,j) for t=0..10:**"))
display(V_df)
display(Markdown("**Exercise decision at each node: E=exercise, C=continue, E/C=indifferent:**"))
display(D_df)

# Find earliest period where exercise (E or E/C) appears
earliest = None
for i in range(1, NO):  # before maturity
    if any(decision[i][j] in ("E", "E/C") for j in range(i+1)):
        earliest = i
        break

earliest


**Futures price lattice F(t,j) for maturity at step 15 (rows t=0..10):**

,j=0,j=1,j=2,j=3,j=4,j=5,j=6,j=7,j=8,j=9,j=10
t=0,100.250313,,,,,,,,,,
t=1,96.425789,104.191792,,,,,,,,,
t=2,92.747170,100.216902,108.288236,,,,,,,,
t=3,89.208889,96.393652,104.157067,112.545737,,,,,,,
t=4,85.805593,92.716259,100.183501,108.252145,116.970627,,,,,,
t=5,82.532131,89.179158,96.361526,104.122354,112.508228,121.569488,,,,,
t=6,79.383551,85.776995,92.685359,100.150113,108.216067,116.931643,126.34916,,,,
t=7,76.355089,82.504625,89.149436,96.329411,104.087652,112.470731,121.528972,131.316751,,,
t=8,73.442162,79.357095,85.748408,92.654469,100.116735,108.180001,116.892673,126.307051,136.47965,,
t=9,70.640362,76.329642,82.477128,89.119725,96.297307,104.052962,112.433247,121.488469,131.272986,141.845535,


**American call on futures: value lattice V(t,j) for t=0..10:**

,j=0,j=1,j=2,j=3,j=4,j=5,j=6,j=7,j=8,j=9,j=10
t=0,1.662673,,,,,,,,,,
t=1,0.746494,2.607994,,,,,,,,,
t=2,0.251124,1.257517,4.001535,,,,,,,,
t=3,0.048037,0.460591,2.079665,5.984884,,,,,,,
t=4,0.000000,0.097575,0.835021,3.363779,8.690196,,,,,,
t=5,0.000000,0.0,0.198199,1.491882,5.295197,12.194897,,,,,
t=6,0.000000,0.0,0.0,0.402593,2.615492,8.060433,16.464051,,,,
t=7,0.000000,0.0,0.0,0.0,0.81777,4.46996,11.76616,21.316751,,,
t=8,0.000000,0.0,0.0,0.0,0.0,1.661101,7.36774,16.307051,26.47965,,
t=9,0.000000,0.0,0.0,0.0,0.0,0.0,3.374124,11.488469,21.272986,31.845535,


**Exercise decision at each node: E=exercise, C=continue, E/C=indifferent:**

,j=0,j=1,j=2,j=3,j=4,j=5,j=6,j=7,j=8,j=9,j=10
t=0,C,,,,,,,,,,
t=1,C,C,,,,,,,,,
t=2,C,C,C,,,,,,,,
t=3,C,C,C,C,,,,,,,
t=4,E/C,C,C,C,C,,,,,,
t=5,E/C,E/C,C,C,C,C,,,,,
t=6,E/C,E/C,E/C,C,C,C,C,,,,
t=7,E/C,E/C,E/C,E/C,C,C,C,E,,,
t=8,E/C,E/C,E/C,E/C,E/C,C,C,E,E,,
t=9,E/C,E/C,E/C,E/C,E/C,E/C,C,E,E,E,


4